In [ ]:
%load_ext autoreload
%autoreload 2
import torch
torch.manual_seed(0) # Set seed before importing other modules
import numpy as np
np.random.seed(0)
import random
random.seed(0)
import sys
import os
from utils.get_data import get_dataloader_augmented, get_dataloader_dino
from training_structures.dino_train import pretrain_dino, train_downstream, train_knn_classifier 
# from models.dino import MultiModalDINO, MultiModalDINOLSTM, MultiModalDINOV2, DownstreamClassifier, dino_loss, dino_loss_align

current_path = os.getcwd()
sys.path.append(current_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ImportError: cannot import name 'MultiModalDINOLSTM' from 'models.dino' (c:\Users\Ward\Desktop\vub-github\Thesis-project\AVMNIST_Experiments\models\dino.py)

In [2]:
dino_model = MultiModalDINOLSTM().to(device)
# save_path = "model_saves/dino/pretrained/pretrained_dino_2025-03-04 17-45-19.pt"
# checkpoint = torch.load(save_path)
# dino_model = MultiModalDINOV2().to(device)
# dino_model.load_state_dict(checkpoint['model_state_dict'])
traindata_dino, _, _ = get_dataloader_dino(
    f'{current_path}/data/avmnist', 
    num_workers=6, 
    type="burst_noise", 
    batch_size=128
)
    
pretrained_model = pretrain_dino(
        dino_model, 
        traindata_dino,
        dino_loss, #TODO: Change to dino_loss_align and align = True, and in the encoder, store self.align = align
        align=False,
        num_epochs=10,
        save_path='model_saves/dino/pretrained/pretrained_dino.pt',
        log_path='training_logs/dino/pretrained/pretrain_log.csv'
    )

Epoch 1/10:   3%|▎         | 13/430 [04:31<2:25:00, 20.87s/it, loss=5.5] 


KeyboardInterrupt: 

In [ ]:
dino_model.student.gate_audio, dino_model.student.gate_image

[autoreload of models.dino failed: Traceback (most recent call last):
  File "c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\Ward\anaconda3\envs\multibench\Lib\importlib\__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "c:\Users\Ward\Desktop\vub-github\Thesis-project\AVMNIST_Experiments\models\dino.py", line 9, in <module>
    from dino_vit import AudioViTEncoder
ModuleNotFoundError: No module named 'dino_vit'
]


(Parameter containing:
 tensor(0.4875, device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor(0.5151, device='cuda:0', requires_grad=True))

In [ ]:
traindata, validdata, testdata = get_dataloader_augmented(f'{current_path}/data/avmnist', type="burst_noise", batch_size=128, num_workers=0)
knn_model, knn_accuracy = train_knn_classifier(pretrained_model, traindata, testdata, n_neighbors=5)

Extracting training features...
Extracting test features...
Training KNN classifier...
KNN Accuracy (k=5): 0.5495


In [ ]:
classifier = train_downstream(
    pretrained_model,
    traindata,
    validdata,
    testdata,
    num_epochs=10,
    device=device,
    save_path='model_saves/dino/downstream/downstream_model.pt',
    train_log_path='training_logs/dino/downstream/downstream_train_log.csv',
    test_log_path='test_logs/dino/downstream/downstream_test_log.csv'
)

Epoch 1/10: 100%|██████████| 430/430 [00:11<00:00, 37.18it/s, loss=1.01] 


Epoch 1: Train Loss: 1.1476, Val Loss: 0.9966, Val Acc: 59.22%
Saved best model with validation accuracy: 59.22%


Epoch 2/10: 100%|██████████| 430/430 [00:11<00:00, 37.89it/s, loss=1.03] 


Epoch 2: Train Loss: 1.0100, Val Loss: 0.9523, Val Acc: 62.26%
Saved best model with validation accuracy: 62.26%


Epoch 3/10: 100%|██████████| 430/430 [00:11<00:00, 37.92it/s, loss=0.968]


...

Epoch 9: Train Loss: 0.9205, Val Loss: 0.8879, Val Acc: 65.42%
Saved best model with validation accuracy: 65.42%


Epoch 10/10: 100%|██████████| 430/430 [00:11<00:00, 36.60it/s, loss=0.848]


Epoch 10: Train Loss: 0.9148, Val Loss: 0.8877, Val Acc: 65.30%


c:\Users\Ward\Desktop\vub-github\Thesis-project\AVMNIST_Experiments\training_structures\dino_train.py:226: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.l


Test Accuracy: 64.06%


In [ ]:
# save_path = "model_saves/dino/pretrained/pretrained_dino_2025-02-25 23-03-44.pt"
# checkpoint = torch.load(save_path)
# pretrained_dino = MultiModalDINO().to(device)
# pretrained_dino.load_state_dict(checkpoint['model_state_dict'])

# traindata, validdata, testdata = get_dataloader_augmented(f'{current_path}/data/avmnist', type="burst_noise", batch_size=128, num_workers=0)
# knn_model, knn_accuracy = train_knn_classifier(pretrained_dino, traindata, testdata, n_neighbors=5)